# RSA Examples in Python

Alice creates a key pair.  She keeps her secret key close to her and sends her public key to Bob

In [8]:
from base64 import b64encode, b64decode
import rsa
(alice_pub,alice_priv) = rsa.newkeys(512)
print (alice_pub)


PublicKey(8083003541550919418819761532127926759551826767490573768945747048150449221013707740401288137796641040609905608912856091389611180409609505269316919643110989, 65537)


Bob wants to send a secret message to Alice. He uses her public key to encrypt the message.
This way he know that ONLY Alice will be able to decrypt it.

In [11]:
message = "Hello Alices".encode('utf8')
private_msg = rsa.encrypt(message,alice_pub)
print(private_msg)

b'hy?H`\xac\xe1\xe7\xc47\xb7-\xf4V\xd8a \x98\x15\xb3\x94"Y\xd2\xe0\xf4yF\x90\xc11\xf8\xf0\xc0\xb0p\xbc\xfb\xc2\xa0\xe9\r\xff\x18wr\x1c\xae\x88\x81I\x11io\x0b\xcb\xfd\xee\x88\x9f.\x0ey\xaa'


Alice receives the `private_msg` and decrypts it with her private key.

In [12]:
plain = rsa.decrypt(private_msg,alice_priv)
print(plain.decode('utf8'))

Hello Alices


Now, for Alice to know that the message came from Bob, Alice wants to use Bob's Public Key to verify the signature of the message (which hopefully Bob was responsible for). So, Bob must not only encrypt his message but also first sign it before sending it to Alice.

In [17]:
(bob_pub,bob_priv) = rsa.newkeys(512)
message = "Hello Alice".encode('utf8')
signature = rsa.sign(message, bob_priv, 'SHA-1')

#message = "Hello Mary".encode('utf8')

sig64 = b64encode(signature)
private_msg = rsa.encrypt(message, alice_pub)
print('Signature Base64 encoded: ', sig64)

Signature Base64 encoded:  b'VXygpAfNw2SdKPw1y/IJyHXm0E2t4QjkOljoRaTAC1pHNN0/dUQnfms65eF/RfkyV8eFYty1mzn5IfGIByhDaw=='


Now when Alice receives this message, (a) only Alice can decrypt the message and (b) Alice can be sure it came from Bob (if and only if she has Bob's Public key directly from Bob or a trusted source)

In [18]:
rawsig = b64decode(sig64)
try:
    bob_is_sender = rsa.verify(message, rawsig, bob_pub)
except Exception as e:
    print('Bob is not the sender!', e)
else:
    print('Bob is sender: ' + bob_is_sender)

Bob is sender: SHA-1


But RSA is using some complex mathematics with large numbers that gets very slow.  Therefore when we encrypt very big files we use RSA to encrypt a block cypher key, and we use that block cypher key to encrypt and decrypt our large file.

---

> `sudo pip3 install cryptography`

In [21]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

key = os.urandom(32)
iv = os.urandom(16)

cipher = Cipher(algorithms.AES(key), modes.CBC(iv))

encryptor = cipher.encryptor()
ct = encryptor.update(b"hello alice     ") + encryptor.finalize()
                        


So RSA would use the `alice_pub` to encrypt the AES `key` and send the encrypted key along with the encrypted message.
Since Alice can decrypt Bob's message she can retrieve the `key` and therefore can decrypt the message.

In [22]:
decryptor = cipher.decryptor()
decryptor.update(ct) + decryptor.finalize()

b'hello alice     '

> Note that the padding on our message is because we are working with raw block ciphers and the AES cipher expects blocks of 16 bytes.